# ハイパーパラメータ 入門編
本資料では、Optunaを使ったブラックボックス最適化を実際に体験していただきます。

# 事前準備 (乱数seedの固定)
Optunaによるブラックボックス最適化は処理の中で乱数が使われるため、通常は実行するたびに結果が変わります。

しかし今回は講義での解説用に、乱数seedを固定して実行結果が毎回同じになるようにしておきます。

以下のプログラムを最初に実行しておいてください。

In [ ]:
import numpy as np
import random

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 100

np.random.seed(SEED)
random.seed(SEED)

もし、これ以降のプログラムを実行したときに「変数SEEDが存在しない」というエラーとなった場合は、上記のプログラムが実行されていないことが原因です。忘れずに実行してください。

# パッケージのインストール
Google Colaboratoryは、標準ではOptunaがインストールされていません。

あらかじめ、pipコマンドでインストールします。

In [ ]:
%pip install optuna

# 簡単な数式の最適化
入門編では、機械学習からはいったん離れて、分かりやすい簡単な数式の最適化を行ってみたいと思います。

## 今回の問題
次に示す関数$f(x, y)$の値が最小となるような$(x, y)$を見つけたいとします。

　　$f(x, y) = (1.5 - x + xy)^2 + (2.25 - x + xy^2)^2 + (2.625 - x + xy^3)^2$

　　$(-4.5 < x < 4.5, -4.5 < y < 4.5)$

　　　※ 本数式は書籍「Optunaによるブラックボックス最適化」より引用しています。

数式が示されているので「ある処理」の中身は分かりますが・・・

問題の答えは簡単には分からないのではないでしょうか。これもある意味、ブラックボックスです。

## Optunaによる最適化プログラム
Optunaを使うと、次のようなプログラムで解を探索することができます。

まずは、細かい内容は気にせずに実行してみましょう。

In [ ]:
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )

# スタディの実行
study.optimize(objective, n_trials=100)

# 結果の表示
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

ログの最後に出力された「Best parameter」が求めた解です。

この問題の最適解は、$(x, y) = (3, 0.5)$ですので、かなり近い値を求めることができました。

プログラム全体で15行程度しかありませんし、なんだか簡単そうだと思いませんか！？

## トライアルとスタディ
プログラムの詳細を解説する前に、Optunaによるブラックボックス最適化における重要な概念である、トライアルとスタディについて説明します。

- トライアル
  - ブラックボックス最適化では、パラメータ値を変化させながら目的関数を繰り返し評価して解を探索します。このプロセスにおける1回の評価を<font color="red">**トライアル(Trial)**</font>と呼びます。

- スタディ
  - トライアルを繰り返す一連の最適化プロセスを、<font color="red">**スタディ(Study)**</font>と呼びます。


<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2024/images/trial_study.png" width="80%" alt="パレートフロントのイメージ">


これらを念頭に置いて、プログラムの内容を見ていきましょう。

## プログラムの解説
プログラムの内容について解説します。次のようなプログラムでしたね。

~~~python
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )

# スタディの実行
study.optimize(objective, n_trials=100)

# 結果の表示
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
~~~

### Optunaパッケージのimport
import文でOptunaのパッケージを読み込みます。

~~~python
import optuna
~~~


### 目的関数の定義
次のコードで、今回の目的関数を定義しています。

~~~python
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2
~~~

Optunaにおいて、目的関数は次のルールで記述します。

- 引数をTrialオブジェクトとする。
  - `trial`という名称で引数を受け取っています。

- 戻り値を評価値とする。
  - `return`文で、戻り値として今回の$f(x, y)$の内容が記述されています。

### サジェストAPI
目的関数内で2回呼び出されている`suggest_float`メソッドは、**<font color="red">サジェストAPI</font>**と呼ばれるものです。

~~~python
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)
~~~

サジェストAPIは、「今回のトライアルではどのパラメータ値を評価するのか」をOptunaが自動的に決めてくれます。

今回の例では、xとyのいずれも-4.5～4.5の浮動小数点数と設定しています。

トライアルごとに、Optunaがこの範囲内で今回のトライアルで試すべきパラメータを提案してくれます。

パラメータ名"x"は"y"は、ログの出力や、結果の可視化で表示するラベルです。


サジェストAPIは他にもありますが、それらは実践編の方でご紹介します。

### スタディの作成
次のコードで、Studyオブジェクトを作成しています。

~~~python
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数を固定するためにSEEDを指定
    )
~~~

Studyオブジェクトの作成には、`create_study`関数を使用します。

- 引数の`direction="minimize"`は、今回の最適化が評価値の最小化を目的とすることを示します。
  - 評価値を最大化するような事例では、`direction="maximize"`と指定します。

- 引数`sampler`では、最適化のアルゴリズム(Optunaではサンプラーと呼びます)として、ベイズ最適化の一つである`TPESampler`を指定しています。
  - `TPESampler`はデフォルトのサンプラーなので通常は指定を省略できますが、今回は乱数seedを固定することで毎回同じ結果とするため、明示的にサンプラーの指定を記述しています。

### スタディの実行
次のコードで、スタディを実行しています。

~~~python
study.optimize(objective, n_trials=100)
~~~

スタディの実行には、Studyオブジェクトの`optimize`メソッドを使用します。

- 引数の`objective`は、定義した目的関数です。
- 引数の`n_trials=100`は、100回のトライアルを実施することを示します。

### 結果の表示
次のコードで、最適化の結果を表示しています。

~~~python
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
~~~

- Studyオブジェクトの`best_value`変数には、求めた解(スタディで試行した全トライアルのうち、最小の評価値)が格納されています。
- Studyオブジェクトの`best_params`には、解が`best_value`となるときのパラメータ値が格納されています。

このようにして、簡単な関数のパラメータについて、ブラックボックス最適化を行うことができました。

Optunaの機能を使えば、ごく簡単なプログラムで実現できることが体感できたのではないでしょうか。

# グリッドサーチやランダムサーチとの比較
今回の事例ではOptunaのデフォルトの手法で最適化を行いました。この手法は、ベイズ最適化によるものです。

同じ事例をグリッドサーチやランダムサーチで行った場合の、解の精度や実行速度を比較してみましょう。

Optunaは、グリッドサーチやランダムサーチの機能も提供しています。

## ベイズ最適化
まずは先ほどと同じベイズ最適化のプログラムについて、以下の処理のみ変更して実行してみます。

- 試行回数を100回から300回に変更
- スタディの実行にかかる時間を計測


In [ ]:
import time
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED) # 乱数を固定するためにSEEDを指定
    )

# スタディの実行
stime = time.perf_counter()             # ★最適化の開始時間を計測
study.optimize(objective, n_trials=300) # ★試行回数を300回に変更
etime = time.perf_counter()             # ★最適化の終了時間を計測

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")  # ★最適化にかかる時間も出力

また、Optunaの可視化機能を使って、最適化の履歴も確認してみましょう。

Optunaが提供する`Visualization`オブジェクトの`plot_optimization_history`メソッドで、各トライアルでの評価値と、その時点でもっとも良い評価値の履歴を出力することができます。

なお可視化機能の詳細は、「ハイパーパラメータ 実践編」でご紹介します。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

評価値の小さいトライアルを集中的に実行できていることが分かります。

なお、右上の凡例「<font color="blue">●</font> Objective Value」をクリックすると、「<font color="red">**ー**</font> Best Value」のみを表示することができます。

最も良い評価値が探索により更新されていくことが分かります。

## グリッドサーチ
Optunaでグリッドサーチを行うには、`create_study`関数で指定する`sampler`をこれまでの`TPESampler`から`GridSampler`に変更します。

~~~python
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.GridSampler(search_space)
    )
~~~

グリッドサーチにおいて、パラメータ値の候補が浮動小数点である場合は、ある程度の間隔で値を区切らなければいけません。

今回は、-4.5から-4.5までを0.1間隔で区切っています。

この場合、xとyの候補の数はそれぞれ91個となりますので、トライアル数は 91 * 91 = 8281 回となります。

プログラムは以下のとおりとなりますが、非常に時間がかかりますので、興味のある方はお時間のあるときに試してみてください。

~~~python
import time
import optuna

# 目的関数
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
           (2.25 - x + x * y ** 2) ** 2 + \
           (2.625 - x + x * y ** 3) ** 2

# GridSampler用のパラメータ範囲と分割数 (0.1間隔)
STEP = 0.1
NUM = int((4.5 - (-4.5)) / step)
search_space = {
    "x": [-4.5 + i * STEP for i in range(NUM+1)],  # -4.5から4.5までを0.1間隔で分割
    "y": [-4.5 + i * STEP for i in range(NUM+1)],
}

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.GridSampler(search_space) # ★グリッドサーチのサンプラーを指定
    )

# スタディの実行
stime = time.perf_counter()
study.optimize(objective, n_trials=len(search_space["x"]) * len(search_space["y"]))
etime = time.perf_counter()

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")
~~~

ちなみに、結果は以下のようになります。

~~~
Best objective value: 0.0
Best parameter: {'x': 3.0, 'y': 0.5}
Time: 24754.8 s
~~~

７時間弱と非常に時間はかかりますが、最適解を含むすべてのパラメータを探索していますので、確実に最適解を求めることができます。

ただし、もしこの問題の最適解が {'x': 3.24, 'y': 0.75} などであった場合はどうなるでしょうか。

今回のパラメータ値の候補にはこれらの値がありませんので、最適解が求められないことになります。

だからといって探索範囲を0.1区切りではなく0.01区切りなどにしたら、どうなるでしょうか。

トライアル数は 901 * 901 = 811801 と100倍近くになり、探索時間も膨大になります。

グリッドサーチは、パラメータの候補が離散的な値であり、その数も少ないケースでのみ、有効な手法といえるでしょう。

【ワンポイント】

もし別の最適化問題で、パラメータの数が5個、10個だったりしたら・・・大変なことになってしまいますね。(これが、次元の罠！)

## ランダムサーチ
Optunaでランダムサーチを行うには、`create_study`関数で指定する`sampler`を`
RandomSampler`に変更します。

~~~python
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.RandomSampler(seed=SEED) # 乱数を固定するためにSEEDを指定
    )
~~~

プログラムは次のとおりであり、サンプラーの指定以外はベイズ最適化のコードと同じです。

こちらも実行してみましょう。

In [ ]:
import time
import optuna

# 目的関数
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
           (2.25 - x + x * y ** 2) ** 2 + \
           (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.RandomSampler(seed=SEED) # ★ランダムサーチのサンプラーを指定
    )

# スタディの実行
stime = time.perf_counter()
study.optimize(objective, n_trials=300)
etime = time.perf_counter()

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")

なんだかちょっとイマイチな結果になりました。yこそまあまあの結果ですが、xは全然ダメですね。

もっともランダムな試行なので、運が悪かったとも言えます。試行回数を多くすれば、最適解にもっと近づくでしょう。

こちらも、最適化がどのように行われたかの履歴を確認してみましょう。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

ランダムなので当然ではありますが、スコアの高くないトライアルも多数行われています。

複雑な最適化問題においては、ベイズ最適化と比べてかなり効率が悪い探索となってしまうことが推測できます。

## 最適化実行時間の比較
実行するタイミングによって結果に幅があるかとは思いますが、こちらで事前に試した結果では、以下となりました。

- グリッドサーチ：24754.8 s
- ランダムサーチ：0.9 s
- ベイズ最適化：4.9 s


傾向として、実行時間の大小が以下の関係であることが確認できました。

　　グリッドサーチ ＞＞ ベイズ最適化 ＞ ランダムサーチ

ランダムサーチとベイズ最適化の比較については、ベイズ最適化自体が計算コストのかかる処理なので、それが原因で差異が出ています。

# おわりに
以上で、「ハイパーパラメータ 入門編」の講義は終了となります。

最初は機械学習からいったん離れて、簡単な関数の最適解を求める問題をOptunaで解いてみました。

自力で作ろうとすると大変そうなブラックボックス最適化の処理も、Optunaを使えば簡単に実現できることが実感できたかと思います。